In [1]:
import os
import rasterio as rio
import numpy as np
import shapely
import pyproj
import geopandas as gpd
import matplotlib.pyplot as plt
import rioxarray as riox
import rasterio as rio
import xarray as xr
import netCDF4
from osgeo import gdal
import pandas as pd
from datetime import datetime
import dask.array
import sklearn
from sklearn.linear_model import LinearRegression

import sys
sys.path.append('../')
import snowFun

In [2]:
# define folder and file paths
folder_AGVA = os.path.join('C:',os.sep,'Users','lzell','OneDrive - Colostate','Desktop',"AGVA")
folder_dems = os.path.join(folder_AGVA, "DEMs", "time_varying_DEMs", "10m")
folder_class = os.path.join(folder_AGVA, 'classified images', 'S2_Classified_Cloudmasked_Merged')
folder_cloud = os.path.join(folder_AGVA, 'classified images', 'S2_Cloud_Merged')
folder_meta = os.path.join(folder_AGVA, "classified images", "meta csv", "S2")
folder_mask = os.path.join(folder_AGVA, 'Derived products', 'S2', 'Masks')
folder_slope = os.path.join(folder_AGVA, 'Derived products', 'S2', 'Slopes')

# open rgi
path_rgi = os.path.join(folder_AGVA, 'RGI', "rgi_2km_o3regions", "rgi_2km_o3regions.shp")
rgi_gdf = gpd.read_file(path_rgi, drop='geometry')

In [3]:
### choose if you want to do only the 45 validation glaciers
validation_only = 1

# open list of validation glaciers
all_validation_df = pd.read_csv(os.path.join(folder_AGVA, 'Validation', 'Validation Glaciers.csv'))

### get list of all the glaciers for which we have calculated the snow covered fractions
# select which rgis to analyze
if validation_only:
    folder_ela = os.path.join(folder_AGVA, 'Derived products', 'S2', 'Validation', 'ELAs')
    folder_save = os.path.join(folder_AGVA, 'Derived products', 'S2', 'Validation')
else:
    folder_sca = os.path.join(folder_AGVA, 'Derived products', 'S2', 'ELAs')
    folder_save = os.path.join(folder_AGVA, 'Derived products', 'S2')

# load rgi names that have been saved to the classified folder
rgis_to_analyze = list(set( [ i[:14] for i in os.listdir(folder_ela) if i!='merged.vrt' ] ))
# rgis_to_analyze.sort()

# get list of glacier area for each rgi
areas = [rgi_gdf[rgi_gdf['RGIId']==i]['Area'].values for i in rgis_to_analyze]

# make df
rgis_to_analyze_df = pd.DataFrame({"RGIId":rgis_to_analyze, 'Area':areas})

# sort however you want
rgis_to_analyze_df = rgis_to_analyze_df.sort_values('Area')

# grab rgi names
rgis_to_analyze = rgis_to_analyze_df['RGIId'].values

print(len(rgis_to_analyze_df))
# print(rgis_to_analyze[:10])
# print(rgis_to_analyze_df[:10])

45


In [6]:
skip = 0
for i in range(len(rgis_to_analyze)):

    # subset rgi to single outline, by choosing rgiid or rgi name
    rgiid = rgis_to_analyze[i]
#     if rgiid!='RGI60-01.15731': continue
        
    rgiid = rgis_to_analyze[i]
    
    # quickly grab glacier area
    ga = rgi_gdf[rgi_gdf['RGIId']==rgiid]['Area'].values[0]

#     if ga>300: continue
    
    # print progress
    print(f"\nStarting {i+1} of {len(rgis_to_analyze)}: {rgiid}  {ga} km2")
    
    # open glacier mask
    glacier_mask = xr.open_dataset(os.path.join(folder_mask, f"S2_{rgiid}_mask.nc"), chunks='auto').glacier
        
    # define the coarsen scale
    if ga>1000:
        scale=5
    elif ga>500:
        scale=3
    else:
        scale=1
        
    if ga>500:
        glacier_mask = glacier_mask.coarsen({"x":scale, "y":scale}, boundary="trim").median(skipna=True).astype('uint8')
        
    # open glacier slope
    glacier_slope = xr.open_dataset(os.path.join(folder_slope, f"S2_{rgiid}_slope.nc"), chunks='auto').slope
    max_slope = 25   
    
    # create binary mask for what is usable slopes
    slope_mask = xr.where(glacier_slope<=max_slope, 1, 0)
    
    # make sure dimensions match up
    slope_mask = slope_mask.reindex_like(glacier_mask, method='nearest')
    
    # coarsen if big glacier
#     glacier_slope = glacier_slope.sel({"x":glacier_mask.x, "y":glacier_mask.y})
    
    # function to format metadata and attributes
    def format_xr_to_save(xr_da):
        xr_da.attrs["res"] = (10*scale,10*scale)
        xr_da.attrs["crs"] = "EPSG:3338"
        xr_da.attrs["transform"] = [10,0,0,0,-10,0]
        xr_da.attrs["_FillValue"] = 0
        xr_da.attrs["long_name"] = rgiid
        xr_da.attrs["description"] = "0: off glacier, no data. 1: ablation. 2: accumulation. 3: on glacier, no data"
        xr_da.name = "accumulation_area"

        xr_da.x.attrs["units"] = "meters"
        xr_da.y.attrs["units"] = "meters"
        xr_da.x.attrs["long_name"] = 'x'
        xr_da.y.attrs["long_name"] = 'y'

        return xr_da
    
    # for each year, open the annual AA product, reformat, save
    for y in [2018,2019,2020,2021,2022]:
        
        # define paths
        aa_path = os.path.join(folder_save, 'Annual AAs', f"S2_{rgiid}_{y}_AA.tif")
        out_path = os.path.join(folder_save, 'Annual AAs', f"S2_{rgiid}_{y}_AA_final.tif")
        
        if os.path.exists(aa_path):
            
            # open
            snow = riox.open_rasterio(aa_path)
            
            # slice slope mask to be the same as snow
#             slope_mask = slope_mask.sel({"x":snow.x, "y":snow.y})

            # give the on-glacier, no data pixels a different value
            snow = xr.where( (snow==0) & (glacier_mask==1), 3, snow  )

            # give on-glacier, steep terrain pixels the same nodata value
            snow = xr.where( (slope_mask==0) & (glacier_mask==1), 3, snow )

            # last formatting
            snow = format_xr_to_save(snow)
            encoding = {"accumulation_area":{"zlib": True}}
            
            # save
            snow.transpose('band', 'y', 'x').rio.to_raster(raster_path=out_path, encoding=encoding)
            
            # remove old
            snow.close()
    
    # lastly, do the same to the 5-year average product
    # open
    aa_path = os.path.join(folder_save, 'Average AAs', f"S2_{rgiid}_2018_2022_average_AA.tif")
    out_path = os.path.join(folder_save, 'Average AAs', f"S2_{rgiid}_2018_2022_average_AA_final.tif")

    if os.path.exists(aa_path):
        snow = riox.open_rasterio(aa_path)

        # give the on-glacier, no data pixels a different value
        snow = xr.where( (snow==0) & (glacier_mask==1), 3, snow  )

        # give on-glacier, steep terrain pixels the same nodata value
        snow = xr.where( (slope_mask==0) & (glacier_mask==1), 3, snow )

        # last formatting
        snow = format_xr_to_save(snow)
        encoding = {"accumulation_area":{"zlib": True}}

        # save
        snow.transpose('band', 'y', 'x').rio.to_raster(raster_path=out_path, encoding=encoding)
        
        # remove old
        snow.close()
        del snow
  
print('All done!')


Starting 1 of 45: RGI60-01.10910  2.084 km2

Starting 2 of 45: RGI60-01.00787  2.126 km2

Starting 3 of 45: RGI60-01.23606  2.344 km2

Starting 4 of 45: RGI60-01.15253  2.551 km2

Starting 5 of 45: RGI60-01.03379  2.578 km2

Starting 6 of 45: RGI60-01.16719  2.681 km2

Starting 7 of 45: RGI60-01.17321  2.88 km2

Starting 8 of 45: RGI60-01.13462  3.206 km2

Starting 9 of 45: RGI60-01.13483  3.216 km2

Starting 10 of 45: RGI60-01.02584  3.441 km2

Starting 11 of 45: RGI60-01.03215  3.998 km2

Starting 12 of 45: RGI60-01.01666  4.243 km2

Starting 13 of 45: RGI60-01.12548  4.314 km2

Starting 14 of 45: RGI60-01.13930  4.404 km2

Starting 15 of 45: RGI60-01.09624  4.487 km2

Starting 16 of 45: RGI60-01.15516  4.764 km2

Starting 17 of 45: RGI60-01.21721  6.422 km2

Starting 18 of 45: RGI60-01.10255  7.262 km2

Starting 19 of 45: RGI60-01.12165  7.969 km2

Starting 20 of 45: RGI60-01.05007  9.216 km2

Starting 21 of 45: RGI60-01.01104  9.528 km2

Starting 22 of 45: RGI60-01.12186  11.05 km

In [4]:
# then go back and delete old files
for i in range(len(rgis_to_analyze)):
    
    # subset rgi to single outline, by choosing rgiid or rgi name
    rgiid = rgis_to_analyze[i]       
    rgiid = rgis_to_analyze[i]
    
    for y in [2018,2019,2020,2021,2022]:

        # define paths
        aa_path =  os.path.join(folder_save, 'Annual AAs', f"S2_{rgiid}_{y}_AA.tif")
        out_path = os.path.join(folder_save, 'Annual AAs', f"S2_{rgiid}_{y}_AA_final.tif")
        
        # if everything looks good, delete
        if os.path.exists(aa_path):
            if os.path.exists(out_path):
                os.remove(aa_path)

    
    # repeat for the average products
    aa_path =  os.path.join(folder_save, 'Average AAs', f"S2_{rgiid}_2018_2022_average_AA.tif")
    out_path = os.path.join(folder_save, 'Average AAs', f"S2_{rgiid}_2018_2022_average_AA_final.tif")
    if os.path.exists(aa_path):
        if os.path.exists(out_path):
            os.remove(aa_path)


In [ ]:
snow